---
title: "Web Scraping"
author: Karisa Kopecek
date: today
format:
  html:
    embed-resources: true
    echo: true
---

# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#pip install beautifulsoup4

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [3]:
# YOUR CODE HERE

URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [ ]:
# YOUR CODE HERE

#response = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")
soup = BeautifulSoup(response.text, "html.parser")


To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [5]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [6]:
len(soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  ))

1

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [7]:
table = soup.find_all("table",
                  attrs={
                       "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                       "style": "text-align:right"}
                  )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [8]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [9]:
city = table.find_all("tr")
len(city)

348

In [10]:
city = table.find_all("tr")[2]
#city, output was too big

In [11]:
cells = city.find_all("td")
#cells, output was too big

In [12]:
#city
name = cells[0].get_text(strip= True)
name

'New York[c]'

In [13]:
#state
state = cells[1].get_text(strip= True)
state

'NY'

In [14]:
 #city, state, population (2022 estimate), and 2020 land area (sq mi)

In [15]:
#population
population = cells[2].get_text(strip= True)
population

'8,478,072'

In [16]:
#land area
area = cells[5].get_text(strip= True)
area

'300.5'

In [17]:
# initialize an empty list
rows = []

# iterate over all rows in the city table
for city in table.find_all("tr")[2:]:

    # Get all the cells (<td>) in the row.
    cells = table.find_all("td")

#see code above
    name = cells[0].find("a")
    name = name.text

    state = cells[1].get_text(strip= True)
    state

    population = cells[2].get_text(strip= True)
    population

    area = cells[5].get_text(strip= True)
    area

    # Append this data.
    rows.append({
        "city": name,
        "state": state,
        "population": population,
        "area": area
    })

In [18]:
pd.DataFrame(rows).head()

,city,state,population,area
0,New York,NY,"8,478,072",300.5
1,New York,NY,"8,478,072",300.5
2,New York,NY,"8,478,072",300.5
3,New York,NY,"8,478,072",300.5
4,New York,NY,"8,478,072",300.5


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [19]:
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

In [20]:
#response.text, output was too big to show

In [21]:
from io import StringIO

df = pd.read_html(StringIO(response.text))

#df, output was too big to show

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [22]:
df2 = pd.read_html(StringIO(response.text), attrs = {"class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                       "style": "text-align:right"})
#df2, output was too big to show

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [23]:
#df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
#df_cities2

df_cities2 = pd.read_html(StringIO(response.text), attrs = {"class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                       "style": "text-align:right"})[0]
#df_cities2, output everything correctly but was very large

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [24]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [25]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [26]:
# Read in the HTML from the URL
URL = "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(URL, headers=HEADERS)

In [27]:
# Parse the HTML with Beautiful Soup
soup = BeautifulSoup(response.text, "html.parser")

In [28]:
# First, find all course blocks
courses = soup.find_all("div", {"class": "courseblock"})
#courses #ran this before, gave the correct output but was super long so put it in hash tags for now

In [29]:
# Looking at the first course block only
course = courses[0]
#course, output was too big to show

In [30]:
# Find the course title
course_title = course.find("p", {"class": "courseblocktitle"})
#course_title, output too big to show

In [31]:
#I want just the title
course_title.get_text(strip=True)

'DATA\xa0100. Data Science for All I.4 units'

In [32]:
# Find the units
units = course_title.find("span", {"class": "courseblockhours"})
units #didn't end up needing this code, looked at hints to remove units

<span class="courseblockhours">4 units
</span>

In [33]:
# Find the div with class="noindent courseextendedwrap" for F,W,S offering
extended_wrap = course.find("div", {"class": "noindent courseextendedwrap"})
#extended_wrap, output too big to show

In [34]:
# Find all tags with class="noindent" inside this div
# The first one is "Term Typically Offered"
term = extended_wrap.find("p", {"class": "noindent"})
term_offered = term.get_text(strip=True)
term_offered

'Term Typically Offered: F, W, SP'

In [35]:
# Now putting it all together

rows = []

# Iterate over all courses
for course in soup.find_all("div", {"class": "courseblock"}):

    # Find the course name
    course_title = course.find("p", {"class": "courseblocktitle"})

     # Remove the span with units, used the hints below to get this
    span = course_title.find("span", {"class": "courseblockhours"})
    if span:
        span.extract()

    course_name = course_title.get_text(strip=True)

    #F,W,S offering
    extended_wrap = course.find("div", {"class": "noindent courseextendedwrap"})

    #need the very first one to get term typically offered
    term = extended_wrap.find("p", {"class": "noindent"})
    term_offered = term.get_text(strip=True)

    # Append data
    rows.append({
        "course": course_name,
        "term_offered": term_offered
    })

#DataFrame
df_courses = pd.DataFrame(rows)
df_courses.head()

,course,term_offered
0,DATA 100. Data Science for All I.,"Term Typically Offered: F, W, SP"
1,DATA 301. Introduction to Data Science.,"Term Typically Offered: F, W, SP"
2,DATA 401. Data Science Process and Ethics.,Term Typically Offered: F
3,DATA 402. Mathematical Foundations of Data Sci...,Term Typically Offered: F
4,DATA 403. Data Science Projects Laboratory.,Term Typically Offered: F


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag